In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from tqdm.notebook import tqdm, trange
import warnings
warnings.filterwarnings("ignore")

In [2]:
circuits = pd.read_csv('../data/circuits.csv')
constructor_results = pd.read_csv('../data/constructor_results.csv')
constructor_standings = pd.read_csv('../data/constructor_standings.csv')
constructors = pd.read_csv('../data/constructors.csv')
driver_standings = pd.read_csv('../data/driver_standings.csv')
drivers = pd.read_csv('../data/drivers.csv', parse_dates=[6])
lap_times = pd.read_csv('../data/lap_times.csv')
pit_stops = pd.read_csv('../data/pit_stops.csv')
qualifying = pd.read_csv('../data/qualifying.csv')
races = pd.read_csv('../data/races.csv', parse_dates=[5])
results = pd.read_csv('../data/results.csv')
seasons = pd.read_csv('../data/seasons.csv')
status = pd.read_csv('../data/status.csv')

In [3]:
result_with_dob = results.merge(drivers, left_on='driverId', right_on='driverId')
results_with_dob_date = result_with_dob.merge(races, on = 'raceId', how = 'left')

In [4]:
#results_with_dob_date['ageAtRace'] = pd.to_datetime(results_with_dob_date['date']) - pd.to_datetime(results_with_dob_date['dob'])
pd.to_datetime(results_with_dob_date['date'])
pd.to_datetime(results_with_dob_date['dob'])

0       1985-01-07
1       1985-01-07
2       1985-01-07
3       1985-01-07
4       1985-01-07
           ...    
24955   1995-06-29
24956   1995-06-29
24957   1995-09-23
24958   1996-06-25
24959   1996-06-25
Name: dob, Length: 24960, dtype: datetime64[ns]

In [5]:
#Calculate age at race in days (timedelta)
results_with_dob_date['ageAtRace'] = results_with_dob_date['date'] - results_with_dob_date['dob']

In [6]:
#Calculate how many days in f1 in days (timedelta)
date_of_debut = results_with_dob_date.groupby('driverId')['date'].min().to_frame()
date_of_debut = date_of_debut.merge(drivers, on = 'driverId', how = 'left').rename(columns={'date':'date_of_debut'})
date_of_debut = date_of_debut[['driverId','date_of_debut']]
date_of_debut['date_of_debut']
df = results_with_dob_date.merge(date_of_debut, on = 'driverId', how = 'left')
df['ageAtDebut'] = df['date_of_debut'] - df['dob']
df['yearsExperience'] = df['date'] - df['date_of_debut']


In [7]:
#Drop columns
df = df.drop(columns=['number_y', 'number_x', 'code', 'forename', 'surname', 'dob', 'url_x', 'time_y', 'url_y'])

In [8]:
#Rename Columns
df = df.rename(columns={"grid": "gridStart", "name": "circuitName", "date_of_debut" : "dateOfDebut", "points": "pointsGained"})

In [9]:
#Add is racing at home feature
df['racingAtHome'] = df.apply(lambda row: row.nationality in row.circuitName, axis=1)

In [10]:
#COnvert ages from days to years
df['ageAtRace'] = df.apply(lambda row: row.ageAtRace / pd.Timedelta('365 days'), axis=1)
df['ageAtDebut'] = df.apply(lambda row: row.ageAtDebut / pd.Timedelta('365 days'), axis=1)
df['yearsExperience'] = df.apply(lambda row: row.yearsExperience / pd.Timedelta('365 days'), axis=1)

In [11]:
#Add starts in front 2 places on the grid
df['startsFrontRow'] = df['gridStart'] <= 2

In [12]:
#Merge with driver standings
driver_standings = driver_standings.rename(columns={"points": "driverStandingsPoints", "position": "driverStandingsPosition", "wins" : "driverStandingsWins"})
driver_standings = driver_standings.drop(columns=['positionText'])
df = df.merge(driver_standings, on = ['raceId','driverId'], how = 'left')

In [13]:
#Get driver standings information before race
df['driverStandingsPoints'] = df.groupby(['year','driverId'])['driverStandingsPoints'].shift(fill_value=0)
df['driverStandingsPosition'] = df.groupby(['year','driverId'])['driverStandingsPosition'].shift(fill_value=0)
df['driverStandingsWins'] = df.groupby(['year','driverId'])['driverStandingsWins'].shift(fill_value=0)

In [14]:
#Merge with constructor standings
constructor_standings = constructor_standings.rename(columns={"points": "constructorStandingsPoints", "position": "constructorStandingsPosition", "wins" : "constructorStandingsWins"})
constructor_standings = constructor_standings.drop(columns=['positionText'])
df = df.merge(constructor_standings, on = ['raceId','constructorId'], how = 'left')

In [15]:
#Get constructor standings information before race
df['constructorStandingsPoints'] = df.groupby(['year','driverId'])['constructorStandingsPoints'].shift(fill_value=0)
df['constructorStandingsPosition'] = df.groupby(['year','driverId'])['constructorStandingsPosition'].shift(fill_value=0)
df['constructorStandingsWins'] = df.groupby(['year','driverId'])['constructorStandingsWins'].shift(fill_value=0)

In [16]:
#Add status for each row
df = df.merge(status, on = ['statusId'], how = 'left')
#Add finished feature
df['finished'] = (df['positionText']) != 'R'

In [17]:
#Get all historic features up until current row date

for date in tqdm(df.date.unique()):
    #Races won by driver
    races_won_by_driver = df[df.date < date].groupby('driverId')['positionText'].apply(lambda x: (x==1).sum()).reset_index(name='racesWon')
    races_won_by_driver['date'] = date
    dfaux = df[df['date'] == date].copy()
    dfaux['racesWon'] = 'to_delete'
    dfaux = dfaux.drop('racesWon', axis = 1)
    df.loc[df['date'] == date, 'racesWon'] = dfaux.merge(races_won_by_driver, on=['date','driverId'], how='left')['racesWon'].values
    #Number of races retired
    races_retired_by_driver = df[df.date < date].groupby('driverId')['positionText'].apply(lambda x: (x=='R').sum()).reset_index(name='racesRetired')
    races_retired_by_driver['date'] = date
    dfaux = df[df['date'] == date].copy()
    dfaux['racesRetired'] = 'to_delete'
    dfaux = dfaux.drop('racesRetired', axis = 1)
    df.loc[df['date'] == date, 'racesRetired'] = dfaux.merge(races_retired_by_driver, on=['date','driverId'], how='left')['racesRetired'].values
    #Number of races finished
    races_finished_by_driver = df[df.date < date].groupby('driverId')['positionText'].apply(lambda x: (x!='R').sum()).reset_index(name='racesFinished')
    races_finished_by_driver['date'] = date
    dfaux = df[df['date'] == date].copy()
    dfaux['racesFinished'] = 'to_delete'
    dfaux = dfaux.drop('racesFinished', axis = 1)
    df.loc[df['date'] == date, 'racesFinished'] = dfaux.merge(races_finished_by_driver, on=['date','driverId'], how='left')['racesFinished'].values
    #Number of pole positions
    pole_positions_by_driver = df[df.date < date].groupby('driverId')['gridStart'].apply(lambda x: (x==1).sum()).reset_index(name='polePositions')
    pole_positions_by_driver['date'] = date
    dfaux = df[df['date'] == date].copy()
    dfaux['polePositions'] = 'to_delete'
    dfaux = dfaux.drop('polePositions', axis = 1)
    df.loc[df['date'] == date, 'polePositions'] = dfaux.merge(pole_positions_by_driver, on=['date','driverId'], how='left')['polePositions'].values


  0%|          | 0/1035 [00:00<?, ?it/s]

In [18]:
#Formating qualifying times
qualifying['q1'] = qualifying['q1'].str.replace('.',':')
qualifying['q2'] = qualifying['q2'].str.replace('.',':')
qualifying['q3'] = qualifying['q3'].str.replace('.',':')

qualifying[['q1', 'q2', 'q3']] = qualifying[['q1', 'q2', 'q3']].applymap(lambda x: pd.to_timedelta(x, errors='coerce'))

qualifying['bestLap'] = qualifying[['q1','q2','q3']].min(axis=1)

In [19]:
#% Diference between drivers qualifying time and best time 
quali_aux = pd.merge(df[['raceId','driverId']], qualifying[['bestLap','raceId','driverId']], on = ['raceId','driverId'], how = 'left')
quali_aux_best = quali_aux.groupby(['raceId'], as_index=False)['bestLap'].min()
df = df.merge(quali_aux, on = ['raceId','driverId'], how = 'left')
df = df.merge(quali_aux_best, on = ['raceId'], how = 'left')
df['percentageOfBestQuali'] = (df['bestLap_x'] / df['bestLap_y'])*100
df = df.drop(columns=['bestLap_x', 'bestLap_y'])

#Other option
#qualifying['diff_from_best'] = qualifying.groupby(['raceId'])['bestLap'].apply(lambda x: x-x.min())
#qualifying['percentageOfBestQuali'] = 

In [25]:
df['gridStart']

0         1
1         9
2         3
3         5
4         3
         ..
25149    16
25150    18
25151    17
25152    20
25153    17
Name: gridStart, Length: 25154, dtype: int64